<div align='center' ><font size='6'>pytorch框架下实现卷积神经网络分类</font></div> 

## 实习要求：

使用pytorch框架的模块完成任务：


**一、使用提供的三种卷积网络形式对HelloRS32数据集分类**
  1. model1：编写并实现简单的双层卷积神经网络的训练及预测，精度达到**86%**以上
  
  
  2. model2：编写并实现三层卷积神经网络的训练及预测，精度达到**88%**以上
      * 使用了relu作为激活函数
      * 在线性层中加入dropout
      * 使用局部响应归一化层lrn
      * 相比与modle1拥有更深的卷积层
      
      
  3. model3：编写并实现很多层的卷积神经网络的训练及预测，精度达到**91%**以上
      * 更小的卷积核尺寸、卷积步长
      * 使用批量归一化batchnorm代替局部响应归一化层lrn
      * 更深的网络
  
  
   4. **选做**：尝试在model3的基础上加入skip connection，进一步提升精度

**二、完成神经网络网络在cifar-10上的训练、预测函数**

# 使用pytorch框架的模块完成任务

Dropout、Batch Norm和2D卷积是计算机视觉中深度学习的主要工具。

在本作业的最后一部分，放弃之前的代码库，转而迁移到流行的深度学习框架之一: PyTorch。

## PyTorch是什么?

PyTorch是一个在Tensor对象上执行动态计算图形的系统，其行为类似于numpy ndarray。它带有一个强大的自动区分引擎，消除了手动反向传播的需要。

### 为什么采用Pytorch

* 我们的代码现在将在gpu上运行速度更快的训练。当使用像PyTorch或TensorFlow这样的框架时，你可以为你自己的自定义神经网络架构利用GPU的能力，而不必直接编写CUDA代码(这超出了本类的范围)。
* 我们希望你准备好在你的项目中使用这些框架之一，这样你就可以更有效地进行试验，而不是手工编写你想要使用的所有功能。
* 我们要你站在巨人的肩膀上！PyTorch是很棒的框架，会让你的生活变得更简单，现在你已经理解了它的本质，你可以自由地使用它了:)
* 我们希望你能接触到你在学术界或工业界可能会遇到的那种深度学习代码。

### PyTorch版本

假设你正在使用**PyTorch 1.0版本**。在之前的一些版本中(比如0.4之前)，Tensor必须被包装在变量对象中才能在autograd中使用；然而变量现在已被弃用。此外，1.0还将Tensor的数据类型从它的设备中分离出来，并使用numpy风格的工厂来构造Tensor，而不是直接调用Tensor构造函数。

## 我将如何学习PyTorch?

[PyTorch 向导](https://github.com/jcjohnson/pytorch-examples) 可以参考github的这些样例。

也可以看Pytorch详细的 [API文档](http://pytorch.org/docs/stable/index.html)。如果你有其他API文档没有解决的问题, [PyTorch forum](https://discuss.pytorch.org/) 也是一个可以参考的地方。


### PyTorch可以分为**三个不同的抽象层次**进行构建：
1. Barebone PyTorch：**抽象级别1**，直接使用最低级别的PyTorch张量。
2. PyTorch模块API：**抽象级别2**，使用`nn.Module`定义任意神经网络结构。
3. PyTorch顺序API：**抽象级别3**，使用`nn.Sequential`可以非常方便地定义线性前馈网络。


以下是一个比较表：

| API           | 灵活性 | 方便性 |
|---------------|-------------|-------------|
| Barebone      |高        | 低         |
| `nn.Module`     | 高       | 中      |
| `nn.Sequential` | 低         | 高        |

# Part I. 准备阶段

首先，加载HelloRS32数据集，对其进行预处理，并以mini_batch遍历它；PyTorch为我们提供了方便的工具来自动化这个过程。

## 导入模块

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time 
from torchvision import transforms
from torchvision.datasets import ImageFolder

## 超参数设置

In [2]:
batch_size = 4  
lr = 0.001 
epoch = 40
print_every = 1000  # 输出损失的频率

## HelloRS32数据集加载

In [3]:
train_path = r"E:\python\jupyter\lab1\Quiz\RS32\train"
val_path = r"E:\python\jupyter\lab1\Quiz\RS32\val"
test_path = r"E:\python\jupyter\lab1\Quiz\RS32\test"
out_dir = r"E:\python\jupyter\lab3\Quiz\model"

data_transform = {
    "train": transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0, 0, 0], [1, 1, 1])]),
    "val": transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0, 0, 0], [1, 1, 1])]),
    "test": transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0, 0, 0], [1, 1, 1])])}

trainset = ImageFolder(train_path, transform=data_transform["train"])
valset = ImageFolder(val_path, transform=data_transform["val"])
testset = ImageFolder(test_path, transform=data_transform["test"])

train_loader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True,num_workers=4)
val_loader = torch.utils.data.DataLoader(valset,batch_size=batch_size,shuffle=True,num_workers=4)
test_loader = torch.utils.data.DataLoader(testset,batch_size=batch_size,shuffle=True,num_workers=4)
print('Done')

Done


你可以**通过设置下面的flag为True使用GPU**。本课程的任务可以不需要使用GPU，但使用GPU可以加速计算。请注意，如果你的计算机没有启用CUDA，`torch.cuda.is_available()`将返回False，将回退到CPU模式。
全局变量`dtype `和`device`将控制整个赋值过程中的数据类型。

In [4]:
dtype = torch.float32  #使用float作为数据类型

USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


# Part II. 模型的设置
新建 `model(*).py`,参照给定的网络构造实现不同卷积神经网络对遥感数据集HelloRS32进行分类：

你需要查看 `torch.nn` 和 `torch.nn.functional` 来了解pytorch中卷积、池化、激活函数、全连接等功能是如何实现的。

当你成功实现model之后，你可以尝试使用更多网络，如`resnet`、`vggnet`、`mobilenet`、`efficientnet`等更强大的网络来在HelloRS32上训练，以提高你的精度，当然这也意味着需要更大的算力

In [6]:
%load_ext autoreload
%autoreload 2
import train
import models
criterion = nn.CrossEntropyLoss()
print('import models done')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
import models done


# Part III. PyTorch的训练

在开始网络的训练之前，我们还需要设置网络的损失函数、优化器，这些设置对于网络的训练也具有重要的影响，但本课程暂不要求对这些设置具有深刻的理解，我们会给出一些示例，你在后续的训练中可以更换不同的损失函数、优化器以及`scheduler`，以提高训练的精度和效率。

In [8]:
net01 = models.Model1().to(device)
print('load model done')
optimizer01 = optim.SGD(net01.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
scheduler01 = torch.optim.lr_scheduler.StepLR(optimizer01, step_size=20, gamma=0.5)
print('preparation done')

load model done
preparation done


完成前期的设置之后，我们就可以开始编写训练的代码了，打开`train.py`，完成`train`和`test`函数，并在下面运行训练和测试的代码。

In [16]:
tic = time.time()
train.train_val(train_loader, val_loader, net01, criterion, optimizer01, epoch, print_every, device, scheduler01,
            output_dir=out_dir+'\01')

toc = time.time()
print('time consuming:',toc - tic)

Epoch: 0
Epoch: 1 | Batch: 1000/1928 | Loss: 1.473 | Acc: 46.125%
val Loss: 1.196 | Acc: 57.974%
Saving..
Epoch: 1
Epoch: 2 | Batch: 1000/1928 | Loss: 1.092 | Acc: 54.095%
val Loss: 1.079 | Acc: 58.573%
Saving..
Epoch: 2
Epoch: 3 | Batch: 1000/1928 | Loss: 0.999 | Acc: 57.508%
val Loss: 0.945 | Acc: 61.005%
Saving..
Epoch: 3
Epoch: 4 | Batch: 1000/1928 | Loss: 0.941 | Acc: 59.808%
val Loss: 0.902 | Acc: 61.842%
Saving..
Epoch: 4
Epoch: 5 | Batch: 1000/1928 | Loss: 0.841 | Acc: 61.583%
val Loss: 0.877 | Acc: 63.206%
Saving..
Epoch: 5
Epoch: 6 | Batch: 1000/1928 | Loss: 0.807 | Acc: 63.204%
val Loss: 0.799 | Acc: 64.740%
Saving..
Epoch: 6
Epoch: 7 | Batch: 1000/1928 | Loss: 0.748 | Acc: 64.488%
val Loss: 0.809 | Acc: 65.710%
Saving..
Epoch: 7
Epoch: 8 | Batch: 1000/1928 | Loss: 0.726 | Acc: 65.614%
val Loss: 0.750 | Acc: 66.766%
Saving..
Epoch: 8
Epoch: 9 | Batch: 1000/1928 | Loss: 0.704 | Acc: 66.561%
val Loss: 0.742 | Acc: 67.588%
Saving..
Epoch: 9
Epoch: 10 | Batch: 1000/1928 | Loss: 

## 测试集

我们通过训练得到了精度最高的模型之后，在测试集上运行我们的模型，并得到测试精度。

In [17]:
#加载模型参数
ckp = r"E:\python\jupyter\lab3\Quiz\model\01\epoch039.pth"
pretrained_dict = torch.load(ckp)
net01.load_state_dict(pretrained_dict["net"])

#测试
train.predict(test_loader, net, criterion, device)

testing Loss: 0.590 | Acc: 83.142%


In [26]:
net02 = models.Model2().to(device)
optimizer02 = optim.SGD(net02.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
scheduler02 = torch.optim.lr_scheduler.StepLR(optimizer02, step_size=20, gamma=0.5)
print('load model done')
tic = time.time()
train.train_val(train_loader, val_loader, net02, criterion, optimizer02, epoch, print_every, device, scheduler02,
            output_dir=out_dir+'\\02')
toc = time.time()
print('time consuming:',toc - tic)

load model done
Epoch: 1 | Batch: 1000/1928 | Loss: 1.695 | Acc: 37.200%
val Loss: 1.016 | Acc: 62.998%
Saving model...
Epoch: 2 | Batch: 1000/1928 | Loss: 0.933 | Acc: 53.873%
val Loss: 0.779 | Acc: 68.740%
Saving model...
Epoch: 3 | Batch: 1000/1928 | Loss: 0.749 | Acc: 60.871%
val Loss: 0.575 | Acc: 71.212%
Saving model...
Epoch: 4 | Batch: 1000/1928 | Loss: 0.603 | Acc: 65.639%
val Loss: 0.504 | Acc: 73.923%
Saving model...
Epoch: 5 | Batch: 1000/1928 | Loss: 0.481 | Acc: 68.742%
val Loss: 0.436 | Acc: 75.901%
Saving model...
Epoch: 6 | Batch: 1000/1928 | Loss: 0.480 | Acc: 70.946%
val Loss: 0.708 | Acc: 75.917%
Saving model...
Epoch: 7 | Batch: 1000/1928 | Loss: 0.421 | Acc: 72.955%
val Loss: 0.588 | Acc: 76.612%
Saving model...
Epoch: 8 | Batch: 1000/1928 | Loss: 0.383 | Acc: 74.627%
val Loss: 0.517 | Acc: 77.163%
Saving model...
Epoch: 9 | Batch: 1000/1928 | Loss: 0.308 | Acc: 76.159%
val Loss: 0.536 | Acc: 77.441%
Saving model...
Epoch: 10 | Batch: 1000/1928 | Loss: 0.276 | Acc

In [12]:
#加载模型参数
ckp = r"E:\python\jupyter\lab3\Quiz\model\02\best_model.pth"
pretrained_dict = torch.load(ckp)
net02.load_state_dict(pretrained_dict["net"])

#测试
train.predict(test_loader, net02, criterion, device)

testing Loss: 0.450 | Acc: 90.803%


In [7]:
net03 = models.ConvNet().to(device)
optimizer03 = optim.SGD(net03.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
scheduler03 = torch.optim.lr_scheduler.StepLR(optimizer03, step_size=20, gamma=0.5)
print('load model done')

tic = time.time()
train.train_val(train_loader, val_loader, net03, criterion, optimizer03, epoch, print_every, device, scheduler03,
            output_dir=out_dir+'\\03')

toc = time.time()
print('time consuming:',toc - tic)

load model done
Epoch: 1 | Batch: 1000/1928 | Loss: 1.682 | Acc: 39.425%
val Loss: 1.048 | Acc: 58.612%
Saving model...
Epoch: 2 | Batch: 1000/1928 | Loss: 1.016 | Acc: 53.531%
val Loss: 0.888 | Acc: 64.474%
Saving model...
Epoch: 3 | Batch: 1000/1928 | Loss: 0.770 | Acc: 60.058%
val Loss: 0.545 | Acc: 70.016%
Saving model...
Epoch: 4 | Batch: 1000/1928 | Loss: 0.609 | Acc: 64.968%
val Loss: 0.556 | Acc: 72.887%
Saving model...
Epoch: 5 | Batch: 1000/1928 | Loss: 0.520 | Acc: 68.461%
val Loss: 0.481 | Acc: 75.247%
Saving model...
Epoch: 6 | Batch: 1000/1928 | Loss: 0.477 | Acc: 71.223%
val Loss: 0.486 | Acc: 76.568%
Saving model...
Epoch: 7 | Batch: 1000/1928 | Loss: 0.403 | Acc: 73.552%
val Loss: 0.420 | Acc: 77.899%
Saving model...
Epoch: 8 | Batch: 1000/1928 | Loss: 0.373 | Acc: 75.314%
val Loss: 0.477 | Acc: 78.668%
Saving model...
Epoch: 9 | Batch: 1000/1928 | Loss: 0.318 | Acc: 76.885%
val Loss: 0.417 | Acc: 79.506%
Saving model...
Epoch: 10 | Batch: 1000/1928 | Loss: 0.284 | Acc

In [8]:
#加载模型参数
ckp = r"E:\python\jupyter\lab3\Quiz\model\03\best_model.pth"
pretrained_dict = torch.load(ckp)
net03.load_state_dict(pretrained_dict["net"])

#测试
train.predict(test_loader, net03, criterion, device)

testing Loss: 0.354 | Acc: 92.705%


In [23]:
net04 = models.Model4().to(device)
optimizer04 = optim.SGD(net04.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
scheduler04 = torch.optim.lr_scheduler.StepLR(optimizer04, step_size=20, gamma=0.5)
print('load model done')

tic = time.time()
train.train_val(train_loader, val_loader, net04, criterion, optimizer04, epoch, print_every, device, scheduler04,
            output_dir=out_dir+'\\04')

toc = time.time()
print('time consuming:',toc - tic)

load model done
Epoch: 1 | Batch: 1000/1928 | Loss: 1.588 | Acc: 44.275%
val Loss: 1.241 | Acc: 63.238%
Saving model...
Epoch: 2 | Batch: 1000/1928 | Loss: 0.837 | Acc: 57.853%
val Loss: 0.778 | Acc: 68.301%
Saving model...
Epoch: 3 | Batch: 1000/1928 | Loss: 0.591 | Acc: 65.208%
val Loss: 0.418 | Acc: 74.428%
Saving model...
Epoch: 4 | Batch: 1000/1928 | Loss: 0.409 | Acc: 70.531%
val Loss: 0.604 | Acc: 76.156%
Saving model...
Epoch: 5 | Batch: 1000/1928 | Loss: 0.312 | Acc: 74.302%
val Loss: 0.406 | Acc: 78.214%
Saving model...
Epoch: 6 | Batch: 1000/1928 | Loss: 0.242 | Acc: 77.264%
val Loss: 0.364 | Acc: 79.745%
Saving model...
Epoch: 7 | Batch: 1000/1928 | Loss: 0.179 | Acc: 79.624%
val Loss: 0.371 | Acc: 80.987%
Saving model...
Epoch: 8 | Batch: 1000/1928 | Loss: 0.152 | Acc: 81.549%
val Loss: 0.313 | Acc: 82.187%
Saving model...
Epoch: 9 | Batch: 1000/1928 | Loss: 0.123 | Acc: 83.162%
val Loss: 0.351 | Acc: 82.881%
Saving model...
Epoch: 10 | Batch: 1000/1928 | Loss: 0.095 | Acc

In [24]:
#加载模型参数
ckp = r"E:\python\jupyter\lab3\Quiz\model\04\best_model.pth"
pretrained_dict = torch.load(ckp)
net04.load_state_dict(pretrained_dict["net"])

#测试
train.predict(test_loader, net04, criterion, device)

testing Loss: 0.233 | Acc: 93.590%
